In [1]:
import sys
sys.path.append('/lmh_data/work/sclab/train/')

import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torch.autograd import Variable
from captum.attr import IntegratedGradients

from dataset import Dataset
from net import define_network

In [2]:
torch.manual_seed(123)
np.random.seed(123)

In [3]:
def evaluate(eval_file, model_file, gene_name):
    eval_set = Dataset(eval_file, gene_name, is_train=False)
    data_loader = data.DataLoader(eval_set, batch_size=1, shuffle=False)

    input_size, output_size = tuple(eval_set[0][0].shape), 171
    patch_size = tuple([int(i / 8) for i in input_size])
    model = torch.nn.DataParallel(define_network(input_size, patch_size, output_size))
    model.load_state_dict(torch.load(model_file))
    model.cuda()
    model.eval()
    ig = IntegratedGradients(model)

    _datas = np.load(eval_file, allow_pickle=True)
    important_genes = dict(ASC=dict(), Endo=dict(), OPC=dict(), ODC=dict())
    for index, batch in enumerate(data_loader, 1):
        if _datas[index-1]['cell_type'] != 'ASC':
            continue
        _important_genes = important_genes[_datas[index-1]['cell_type']]
        input = Variable(batch[0]).cuda().unsqueeze(1)
#         target = Variable(batch[1]).cuda().unsqueeze(1) / 10
        target = model(input)
        
        attributions = torch.zeros(input.reshape(-1).shape).cuda()
        for i in range(target.shape[-1]):
            attributions += ig.attribute(input, target=(0, i))[0, 0].reshape(-1)
        attributions = attributions.cpu().detach().numpy()
        for i in range(len(attributions)):
            if i not in _important_genes.keys():
                _important_genes[i] = 0
            _important_genes[i] += attributions[i]
        
    return important_genes

important_genes = evaluate(
    '/lmh_data/data/sclab/sclab/AD/eval_dataset.npy',
    '/lmh_data/data/sclab/sclab/AD/tmp/SLC1A2/model_epoch_8.pth', 'SLC1A2')
important_genes

{'ASC': {0: -0.09720388087589527,
  1: -0.0170873305760324,
  2: 0.0,
  3: -0.02247831365093589,
  4: 0.0,
  5: -0.45267390459775925,
  6: 0.037893409142270684,
  7: -0.015868610935285687,
  8: 0.0,
  9: 0.00256787336547859,
  10: -0.679028568090871,
  11: -0.16904908511787653,
  12: 0.016479189667734317,
  13: -0.0034834062680602074,
  14: 0.029770877212285995,
  15: -0.005074938177131116,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: 0.03938230021594791,
  20: -0.8290733606554568,
  21: 0.0,
  22: 0.08521550195291638,
  23: 0.0,
  24: 0.0,
  25: 0.005593040958046913,
  26: 0.7580754664959386,
  27: 0.008770793676376343,
  28: 0.0,
  29: 0.0,
  30: -0.5833392235217616,
  31: -0.5843606004491448,
  32: 0.0,
  33: 0.07804385735653341,
  34: 0.0,
  35: -0.0936693362891674,
  36: 0.00656997982878238,
  37: -0.005169018637388945,
  38: 0.0,
  39: -3.236746064387262,
  40: 0.0,
  41: 0.0,
  42: 0.14874877920374274,
  43: -0.11830547964200377,
  44: -0.16598511304619024,
  45: 0.0,
  46: -0.3714647

In [4]:
scRNA_head = np.load('/lmh_data/data/sclab/sclab/AD/eval_dataset.npy', allow_pickle=True)[0]['scRNA_head']

def find_special_genes(important_genes, cell_type):
    genes = pd.DataFrame(
        sorted(important_genes[cell_type].items(), key=lambda x: x[1], reverse=True), columns=['index', 'value']
    ).set_index('index')
    genes = genes.sort_values(by=['value'], ascending=False)
    return genes

genes = find_special_genes(important_genes, 'ASC')
genes['name'] = scRNA_head[list(genes.index)]
genes.to_csv('tmp/related_genes_SLC1A2.csv')
genes

,value,name
index,,
12171,55.023629,LRP1B
16466,42.754380,ZBTB20
3904,40.076972,SLC1A2
11179,28.675374,NRG3
8600,25.728704,PRKCA
...,...,...
7268,-25.667883,TNRC6A
16436,-26.335460,GABRB1
7532,-27.778125,PITPNC1
